## Scam online dating profile download script

The below script has been sourced, with amendments, from the following repository: https://github.com/gsuareztangil/automatic-romancescam-digger. 

In [1]:
import os
import re
import json
import time
import hashlib
import random
from bs4 import BeautifulSoup
from urllib.request import urlopen

IMAGEDIR='images_scam_new'
PROFILES='dating_scam_new'

In [2]:
# Descriptions has been commented out, as this is not required for my project. 
# A few amendments have been made from the original code, to generalise for a change in format to the format on scamdiggers.com.
# This was a change to use capitalised titles and removed the ':' from the field. 

extractors = {'username': re.compile('[Uu]sername[:]* ([^\n]+)'),
              'name': re.compile('\W[Nn]ame[:]* ([^\n]+)'),
              'age': re.compile('\W[Aa]ge[:]* ([^\n]+)'),
              'city': re.compile('\W[Cc]ity[:]* ([^\n]+)'),
              'country': re.compile('\W[Cc]ountry[:]* ([^\n]+)'),
              'location': re.compile('\W[Ll]ocation[:]* ([^\n]+)'),
              'ethnicity': re.compile('\W[Ee]thni?city[:]* ([^\n]+)'),
              'occupation': re.compile('\W[Oo]ccupation[:]* ([^\n]+)'),
              'status': re.compile('\W[Mm]arital status[:]* ([^\n]+)'),
              'phone': re.compile('\W[Tt]el[:]* ([^\n]+)'),
              'inet': re.compile('\WIP address[:]* ([^\n]+)'),
              'email': re.compile('\W[Ee]mail[:]* ([^\n]+)'),
#               'description': re.compile('\W[Dd]escription[:]*([\n\w\W]+)\W[Mm]essage[:]*'),
              'messages': re.compile('\W[Mm]essage[:]*([\n\w\W]+)\WWHY IS'),
              'justifications': re.compile('\WWHY IS IT A SCAM / FAKE[:]*([\n\w\W]+)\W This post')}

In [4]:
def save_image(url):
    """ Take a URL, generate a unique filename, save 
        the image to said file and return the filename."""
    
    ext = url.split('.')[-1]
    filename = os.getcwd()+os.sep+IMAGEDIR+os.sep+hashlib.md5(url.encode('utf-8')).hexdigest()+'.'+ext
    
    if os.path.exists(filename):
        return filename
    try:
        content = urlopen(url).read()
        f = open(filename,'wb') 
        f.write(content)
        f.close()
    except:
        return None
    return filename 

In [5]:
def scrape_profile(inhandle, outfile, year, month):
    """Scrape an input scamdiggers page for the profile content
    of the scammer. """
    
    # Read file.
    html = inhandle.read()
    soup = BeautifulSoup(html, 'html.parser')

    # Find main page content.
    content = soup.find('div', {'class':'entry-content'})

    profile = {}

    # Fill in known info from URL.
    profile['year_reported'] = year
    profile['month_reported'] = month

    #Extract and download images.
    """""Images not used in the end for the project, but this is an option."""
    
    #profile['images'] = [save_image(img['src']) for img in content.findAll('img')]

    # Get visible text.
    text = content.get_text().strip()

    # Parse information from text.
    for key in extractors:
        match = extractors[key].search(text)
        if match:
            matchtext = match.group(1).strip()
            if key in ['justifications','messages']:
                vals = matchtext.split('\n')
            else:
                vals = matchtext
            profile[key] = vals 

    # Parse annotations.
    content = soup.find('div', {'class':'entry-utility'})
    profile['tags']   = [node.get_text() for node in content.findAll('a', {'rel':'tag'})]
    profile['gender'] = 'female' if 'Female profiles' in profile['tags'] else 'male'

    # Save output.
    json.dump(profile, open(outfile,'w'))

In [6]:
def enumerate_profiles(inhandle, page):
    """ Extract all the profile page links from
    this index page. """
    
    soup = BeautifulSoup(inhandle.read(), 'html.parser')
    urls = []
    for node in soup.findAll('h1',  {'class':'entry-title'}):
        try:
            urls.append(node.find('a')['href'])
        except:
            pass
    return urls

In [7]:
def gather_all_profiles(year, month):
    """ Walk the index pages, harvesting the profile URLs,
    and then download and process all the profiles stored 
    under this year and month. """
    
    page = 1
    urls = []

    print("{}-{} : Begin indexing.".format(year, month))

    while (page > 0):
        urlstring = "http://scamdigger.com/{}/{}/page/{}".format(year,month,page)    
        jitter = random.choice([0,1])
        try:
            urlhandle = urlopen(urlstring)
            urls += enumerate_profiles(urlhandle, page)
            time.sleep(1+jitter)
            page += 1
        except:
            page = 0

    print("{}-{} : {} profiles".format(year,month,len(urls)))

    for url in urls:
        uid = url[31:-1]
        outfile=os.getcwd()+os.sep+PROFILES+os.sep+uid+'.json'
        jitter = random.choice([0,1])
        try:
            urlhandle = urlopen(url)
            scrape_profile(urlhandle, outfile, year, month)
            time.sleep(1+jitter)
        except Exception as e:
            print("Exception when handling {}".format(url))
            print(e)

    print("{}-{} : complete.".format(year,month))

In [8]:
def scrape(startyear, startmonth, endyear, endmonth):
    """ Walk the database through the defined ranges,
    downloading everything. """
    
    year = startyear
    month = startmonth
    while (not (year == endyear and month == endmonth)):
        ys = "{}".format(year)
        ms = "{:02d}".format(month)
        gather_all_profiles(ys,ms) 
        if month == 12:
            year += 1
            month = 0
        month += 1

In [9]:
# Use this function to scrape the scam profiles.
# Specify the date range.

scrape(2012,5,2022,8)

2012-05 : Begin indexing.
2012-05 : 30 profiles
2012-05 : complete.
2012-06 : Begin indexing.
2012-06 : 116 profiles
2012-06 : complete.
2012-07 : Begin indexing.
2012-07 : 68 profiles
2012-07 : complete.
2012-08 : Begin indexing.
2012-08 : 45 profiles
2012-08 : complete.
2012-09 : Begin indexing.
2012-09 : 31 profiles
2012-09 : complete.
2012-10 : Begin indexing.
2012-10 : 14 profiles
2012-10 : complete.
2012-11 : Begin indexing.
2012-11 : 34 profiles
2012-11 : complete.
2012-12 : Begin indexing.
2012-12 : 35 profiles
2012-12 : complete.
2013-01 : Begin indexing.
2013-01 : 9 profiles
2013-01 : complete.
2013-02 : Begin indexing.
2013-02 : 8 profiles
2013-02 : complete.
2013-03 : Begin indexing.
2013-03 : 19 profiles
2013-03 : complete.
2013-04 : Begin indexing.
2013-04 : 20 profiles
2013-04 : complete.
2013-05 : Begin indexing.
2013-05 : 75 profiles
2013-05 : complete.
2013-06 : Begin indexing.
2013-06 : 20 profiles
2013-06 : complete.
2013-07 : Begin indexing.
2013-07 : 17 profiles
2

2022-05 : complete.
2022-06 : Begin indexing.
2022-06 : 1 profiles
2022-06 : complete.
2022-07 : Begin indexing.
2022-07 : 4 profiles
2022-07 : complete.
